# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

First, we import everything we will need later.

In [68]:
#imports
import pandas as pd
import datetime
from dateutil.parser import parse
from os import listdir
from os.path import isfile, join

Then, we define some helpful functions that will help us during the parsing of the data.
- get_files: returns all the .csv files for a given country

In [69]:
def get_files(country):
    path = DATA_FOLDER + "/ebola/" + country + "_data/"
    return [f for f in listdir(path) if isfile(join(path, f))]


- sum_row: for a given row, returns the total value for the new cases / deaths. We first defined this function as the sum of all new cases / deaths in all provinces, but we discovered some strange data for some provinces, so we decided to only take into account the 'total' column
- sum_rows: sum all the rows given in argument

In [70]:
def sum_row(row, drop_col, total_col):
    return float(row[total_col].values[0])
    #return sum(row.drop(row.columns[drop_col], axis=1).fillna(0.0).apply(lambda x: float(x)))

def sum_rows(rows, drop_col, total_col):
    tot = 0
    for row in rows:
        tot += sum_row(row, drop_col, total_col)
    return tot 

Now, we define for each country a function, which, for a given file, returns a dictionnary with the country, date, upper and lower bounds for the new cases, and upper and lower bounds for the new deaths.
As we don't know if the new cases / deaths for the 'probable' and 'suspected' cases is reliable, we decided to create an upper bound with the sum of the 'confirmed', 'probable' and 'suspected' new cases / deaths, and a lower bound with only the 'confirmed' new cases / deaths.

The structure of these functions are the same for each country, only the name of the descrption of the data changes.

In [75]:
def get_row_guinea(country, file):
    date = file[:10]
    raw = pd.read_csv(DATA_FOLDER + "/ebola/" + country + "_data/" + file)
    drop_col = range(3)
    total_col = "Totals"
    
    new_cases_lower = sum_row(raw[raw.Description == "New cases of confirmed"], drop_col, total_col)
    new_cases_upper = sum_row(raw[raw.Description == "Total new cases registered so far"], drop_col, total_col)
        
    new_deaths_lower = sum_row(raw[(raw.Description == "New deaths registered today (confirmed)") | (raw.Description == "New deaths registered")], drop_col, total_col)
    new_deaths_upper = sum_row(raw[(raw.Description == "New deaths registered today") | (raw.Description == "New deaths registered")], drop_col, total_col)
    
    return {'Country' : country, 'Date' : parse(date), 'NewCasesLower' : new_cases_lower, 'NewCasesUpper' : new_cases_upper, 'NewDeathsLower' : new_deaths_lower, 'NewDeathsUpper' : new_deaths_upper}

In [76]:

def get_row_liberia(country, file):
    date = file[:10]
    raw = pd.read_csv(DATA_FOLDER + "/ebola/" + country + "_data/" + file).fillna(0)
    drop_col = range(3)
    total_col = "National"
    
    new_cases_lower = sum_row(raw[raw.Variable == "New case/s (confirmed)"], drop_col, total_col)
    list_cases_upper = (["New Case/s (Suspected)", 
                        "New Case/s (Probable)",
                        "New case/s (confirmed)"])
    new_cases_upper = sum_rows([raw[raw.Variable == row] for row in list_cases_upper], drop_col, total_col)
        
    new_deaths_lower = sum_row(raw[raw.Variable == "Newly reported deaths"], drop_col, total_col)
    new_deaths_upper = new_deaths_lower
    
    return {'Country' : country, 'Date' : parse(date), 'NewCasesLower' : new_cases_lower, 'NewCasesUpper' : new_cases_upper, 'NewDeathsLower' : new_deaths_lower, 'NewDeathsUpper' : new_deaths_upper}

As the files for the Sierra Leone does not contain data for the new deaths, we first extract the total deaths for each day, and we will process them later to get the new deaths.

In [77]:
def get_row_sl(country, file):
    date = file[:10]
    raw = pd.read_csv(DATA_FOLDER + "/ebola/" + country + "_data/" + file).fillna(0)
    drop_col = [0,1,-1]
    total_col = "National"
        
    new_cases_lower = sum_row(raw[raw.variable == "new_confirmed"], drop_col, total_col)
    list_cases_upper = (["new_suspected", 
                        "new_probable",
                        "new_confirmed"])
    new_cases_upper = sum_rows([raw[raw.variable == row] for row in list_cases_upper], drop_col, total_col)
    
    list_death_upper = (["death_suspected", 
                        "death_probable",
                        "death_confirmed"])
    total_death_upper = sum_rows([raw[raw.variable == row] for row in list_death_upper], drop_col, total_col)
    total_death_lower = sum_row(raw[raw.variable == "death_confirmed"], drop_col, total_col)
    
    return {'Country' : country, 'Date' : parse(date), 'NewCasesLower' : new_cases_lower, 'NewCasesUpper' : new_cases_upper, 'TotalDeathLower' : total_death_lower, 'TotalDeathUpper' : total_death_upper}

In [78]:
rows_guinea = [get_row_guinea("guinea", file) for file in get_files("guinea")]

In [79]:
rows_liberia = [get_row_liberia("liberia", file) for file in get_files("liberia")]

We now transform the data for the Sierra Leone :
-we first create a new dictionary for which the keys are date, and the values are the previously extracted values from the .csv files.
-then for each value in this dictionary, we try to get the value of the day before, and perform the difference to get the new deaths of this day

In [80]:
rows_sl_total_deaths = [get_row_sl("sl", file) for file in get_files("sl")]
dic_sl_total_deaths = {}
for row in rows_sl_total_deaths:
    dic_sl_total_deaths[row['Date']] = row
    
rows_sl = []
for date, entry in dic_sl_total_deaths.items():
    date_before = date - datetime.timedelta(days=1)
    if date_before in dic_sl_total_deaths:
        
        if entry['TotalDeathUpper'] != 0 and dic_sl_total_deaths[date_before]['TotalDeathUpper'] != 0 and entry['TotalDeathLower'] != 0 and dic_sl_total_deaths[date_before]['TotalDeathLower'] != 0: 
            copy = dict(entry)
            del copy['TotalDeathUpper']
            del copy['TotalDeathLower']
            
            copy['NewDeathsUpper'] = entry['TotalDeathUpper'] - dic_sl_total_deaths[date_before]['TotalDeathUpper']
            copy['NewDeathsLower'] = entry['TotalDeathLower'] - dic_sl_total_deaths[date_before]['TotalDeathLower']

            rows_sl.append(copy)

We can now insert the data in a dataframe

In [84]:
raw_dataframe = pd.DataFrame(columns=['Country', 'Date', 'NewCasesLower', 'NewCasesUpper', 'NewDeathsLower', 'NewDeathsUpper'])
for row in rows_sl, rows_guinea:
    raw_dataframe = raw_dataframe.append(row, ignore_index = True)
for row in rows_liberia:
    if row['Date'].month != 12: #December data is erroneous
        raw_dataframe = raw_dataframe.append(row, ignore_index = True)
        
raw_dataframe

,Country,Date,NewCasesLower,NewCasesUpper,NewDeathsLower,NewDeathsUpper
0,sl,2014-08-24,31.0,31.0,5.0,5.0
1,sl,2014-08-23,23.0,27.0,3.0,6.0
2,sl,2014-10-24,41.0,54.0,14.0,42.0
3,sl,2014-10-31,95.0,124.0,6.0,6.0
4,sl,2014-09-28,90.0,109.0,4.0,4.0
5,sl,2014-11-08,111.0,131.0,7.0,7.0
6,sl,2014-11-23,39.0,64.0,31.0,31.0
7,sl,2014-09-09,36.0,42.0,3.0,3.0
8,sl,2014-09-15,29.0,33.0,6.0,6.0
9,sl,2014-10-05,48.0,60.0,21.0,45.0


In [ ]:
dataframe = raw_dataframe.set_index(['Country', 'Date'])

In [85]:
dataframe_no_day = raw_dataframe
dataframe_no_day['Year'] = raw_dataframe['Date'].apply(lambda x: x.year)
dataframe_no_day['Month'] = raw_dataframe['Date'].apply(lambda x: x.month)
dataframe_no_day[['Country', 'Year', 'Month', 'NewCasesLower', 'NewCasesUpper', 'NewDeathsLower', 'NewDeathsUpper']].groupby(['Country', 'Year', 'Month']).mean()

NewCasesLower  NewCasesUpper  NewDeathsLower  \
Country Year Month                                                 
guinea  2014 8          12.400000      25.800000        3.400000   
             9          13.000000      19.625000        3.562500   
             10          6.000000      34.000000       15.000000   
liberia 2014 6           2.142857       5.714286        2.000000   
             7           1.818182       8.545455        4.272727   
             8           5.444444      37.222222       23.222222   
             9           6.166667      63.833333       36.041667   
             10          1.360000      45.560000       28.040000   
             11          2.600000      26.466667       13.466667   
sl      2014 8          19.266667      26.666667        6.400000   
             9          37.360000      44.160000        5.120000   
             10         59.956522      73.652174       13.086957   
             11         63.846154      78.153846       16.076923   
             12         27.000000      39.000000       30.000000   

                    NewDeathsUpper  
Country Year Month                  
guinea  2014 8            3.400000  
             9            3.562500  
             10          15.000000  
liberia 2014 6            2.000000  
             7            4.272727  
             8           23.222222  
             9           36.041667  
             10          28.040000  
             11          13.466667  
sl      2014 8            6.800000  
             9            5.120000  
             10          23.391304  
             11          16.076923  
             12          30.000000

In [ ]:
# Write your answer here

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here